In [29]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [3]:
a=[[1],[2]];b=[[3],[4]];c=a+b;print(c);
data=[]
for tpl in c:data=data+tpl
print(data)

[[1], [2], [3], [4]]
[1, 2, 3, 4]


In [ ]:
# from `sigma_0 experiments13may23'
# output snipped.

import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=0) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=10000)
import pickle
datafile= open('/Users/barrybrent/run14may23.txt','wb') 
data=[]
start=time.time()/60
for a in [0..2000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

In [3]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
print(s[0]);print(s[1]);
print(len(s))

(0, 1)
(1, -2)
2001


In [15]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)


for k in [0..50]:
    pair=s[k]
    a=pair[0]
    m3=ZZ(mod(a,3))
    print((a-m3))

0
0
0
3
3
3
6
6
6
9
9
9
12
12
12
15
15
15
18
18
18
21
21
21
24
24
24
27
27
27
30
30
30
33
33
33
36
36
36
39
39
39
42
42
42
45
45
45
48
48
48


In [18]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..50]:
    pair=s[k]
    a=pair[0]
    m3=ZZ(mod(a,3)) #seem to need the ZZ to avoid errors
    om=a-m3
    c=pair[1]
    o2=order(2,c)
    print((a,order(3,om),o2))
    data=data+[o2]
print(data)   

(0, +Infinity, 0)
(1, +Infinity, 1)
(2, +Infinity, 3)
(3, 1, 0)
(4, 1, 3)
(5, 1, 2)
(6, 1, 0)
(7, 1, 2)
(8, 1, 3)
(9, 2, 0)
(10, 2, 3)
(11, 2, 1)
(12, 1, 0)
(13, 1, 3)
(14, 1, 3)
(15, 1, 0)
(16, 1, 3)
(17, 1, 1)
(18, 2, 0)
(19, 2, 4)
(20, 2, 3)
(21, 1, 0)
(22, 1, 6)
(23, 1, 2)
(24, 1, 0)
(25, 1, 2)
(26, 1, 2)
(27, 3, 0)
(28, 3, 5)
(29, 3, 2)
(30, 1, 0)
(31, 1, 1)
(32, 1, 3)
(33, 1, 0)
(34, 1, 2)
(35, 1, 2)
(36, 2, 0)
(37, 2, 1)
(38, 2, 5)
(39, 1, 0)
(40, 1, 3)
(41, 1, 4)
(42, 1, 0)
(43, 1, 1)
(44, 1, 4)
(45, 2, 0)
(46, 2, 2)
(47, 2, 1)
(48, 1, 0)
(49, 1, 1)
(50, 1, 2)
[0, 1, 3, 0, 3, 2, 0, 2, 3, 0, 3, 1, 0, 3, 3, 0, 3, 1, 0, 4, 3, 0, 6, 2, 0, 2, 2, 0, 5, 2, 0, 1, 3, 0, 2, 2, 0, 1, 5, 0, 3, 4, 0, 1, 4, 0, 2, 1, 0, 1, 2]


In [25]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

data=[]
for k in [0..199]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if o3==0:
        print((a,a.digits(3)))

(0, [])
(1, [1])
(2, [2])
(3, [0, 1])
(4, [1, 1])
(5, [2, 1])
(6, [0, 2])
(7, [1, 2])
(9, [0, 0, 1])
(11, [2, 0, 1])
(12, [0, 1, 1])
(15, [0, 2, 1])
(16, [1, 2, 1])
(18, [0, 0, 2])
(21, [0, 1, 2])
(22, [1, 1, 2])
(23, [2, 1, 2])
(26, [2, 2, 2])
(27, [0, 0, 0, 1])
(31, [1, 1, 0, 1])
(33, [0, 2, 0, 1])
(34, [1, 2, 0, 1])
(35, [2, 2, 0, 1])
(36, [0, 0, 1, 1])
(37, [1, 0, 1, 1])
(38, [2, 0, 1, 1])
(40, [1, 1, 1, 1])
(41, [2, 1, 1, 1])
(44, [2, 2, 1, 1])
(45, [0, 0, 2, 1])
(46, [1, 0, 2, 1])
(47, [2, 0, 2, 1])
(48, [0, 1, 2, 1])
(50, [2, 1, 2, 1])
(52, [1, 2, 2, 1])
(53, [2, 2, 2, 1])
(54, [0, 0, 0, 2])
(56, [2, 0, 0, 2])
(58, [1, 1, 0, 2])
(62, [2, 2, 0, 2])
(63, [0, 0, 1, 2])
(64, [1, 0, 1, 2])
(65, [2, 0, 1, 2])
(66, [0, 1, 1, 2])
(68, [2, 1, 1, 2])
(69, [0, 2, 1, 2])
(70, [1, 2, 1, 2])
(71, [2, 2, 1, 2])
(74, [2, 0, 2, 2])
(76, [1, 1, 2, 2])
(77, [2, 1, 2, 2])
(78, [0, 2, 2, 2])
(79, [1, 2, 2, 2])
(80, [2, 2, 2, 2])
(81, [0, 0, 0, 0, 1])
(83, [2, 0, 0, 0, 1])
(86, [2, 1, 0, 0, 1])
(88, 

In [8]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..1999]:
    pair=s[k]
    a=pair[0]
    if mod(a,3)==0:
        c=pair[1]
        o2=order(2,c)
        if o2==0:yes=yes+1
        if o2!=0:no=no+1
print((yes,no))

(667, 0)


In [20]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)


for k in [0..199]:
    pair=s[k]
    a=pair[0]
   
    c=pair[1]
    o3=order(3,c)
    print((a,o3))
print((yes,no))

(0, 0)
(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 1)
(9, 0)
(10, 1)
(11, 0)
(12, 0)
(13, 2)
(14, 1)
(15, 0)
(16, 0)
(17, 2)
(18, 0)
(19, 1)
(20, 1)
(21, 0)
(22, 0)
(23, 0)
(24, 2)
(25, 1)
(26, 0)
(27, 0)
(28, 1)
(29, 3)
(30, 2)
(31, 0)
(32, 1)
(33, 0)
(34, 0)
(35, 0)
(36, 0)
(37, 0)
(38, 0)
(39, 1)
(40, 0)
(41, 0)
(42, 1)
(43, 1)
(44, 0)
(45, 0)
(46, 0)
(47, 0)
(48, 0)
(49, 1)
(50, 0)
(51, 1)
(52, 0)
(53, 0)
(54, 0)
(55, 2)
(56, 0)
(57, 1)
(58, 0)
(59, 2)
(60, 1)
(61, 2)
(62, 0)
(63, 0)
(64, 0)
(65, 0)
(66, 0)
(67, 1)
(68, 0)
(69, 0)
(70, 0)
(71, 0)
(72, 2)
(73, 2)
(74, 0)
(75, 2)
(76, 0)
(77, 0)
(78, 0)
(79, 0)
(80, 0)
(81, 0)
(82, 2)
(83, 0)
(84, 1)
(85, 1)
(86, 0)
(87, 3)
(88, 0)
(89, 0)
(90, 2)
(91, 0)
(92, 3)
(93, 0)
(94, 2)
(95, 0)
(96, 1)
(97, 0)
(98, 2)
(99, 0)
(100, 0)
(101, 0)
(102, 0)
(103, 1)
(104, 0)
(105, 0)
(106, 0)
(107, 0)
(108, 0)
(109, 0)
(110, 2)
(111, 0)
(112, 2)
(113, 0)
(114, 0)
(115, 0)
(116, 3)
(117, 1)
(118, 0)
(119, 0)
(120, 0)
(121, 0)
(122, 1)
(12

In [21]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0

for k in [0..199]:
    pair=s[k]
    a=pair[0]
    if ZZ(mod(a,9))==8:
        c=pair[1]
        o3=order(3,c)
        if o3==1:yes=yes+1
        if o3!=1:no=no+1
print((yes,no))

(4, 18)


In [9]:
data2=[]
data3=[]
for j in [0..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(1)
    o2=order(2,rsp)
    data2=data2+[o2]
    o3=order(3,rsp)
    data3=data3+[o3]
print(data2);print();print(data3)



[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 4, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 5, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0]


In [15]:

data2=[]
data3=[]
for j in [0..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(1)
   
    o3=order(3,rsp)
    if mod(j,2)==0:
        print((j,o3))
        


(0, 0)
(2, 0)
(4, 0)
(6, 0)
(8, 0)
(10, 0)
(12, 0)
(14, 0)
(16, 0)
(18, 0)
(20, 0)
(22, 0)
(24, 0)
(26, 0)
(28, 0)
(30, 0)
(32, 0)
(34, 0)
(36, 0)
(38, 0)
(40, 0)
(42, 0)
(44, 0)
(46, 0)
(48, 0)
(50, 0)
(52, 0)
(54, 0)
(56, 0)
(58, 0)
(60, 0)
(62, 0)
(64, 0)
(66, 0)
(68, 0)
(70, 0)
(72, 0)
(74, 0)
(76, 0)
(78, 0)
(80, 0)
(82, 0)
(84, 0)
(86, 0)
(88, 0)
(90, 0)
(92, 0)
(94, 0)
(96, 0)
(98, 0)
(100, 0)


In [21]:
data=[]
for j in [2..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(1)
   
    o3=order(3,rsp)
    if mod(j,2)==1:
        data=data+[order(3,j)+1-o3]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [23]:
data=[]
for j in [2..100]:
    def Sigma(n):
        ans=0
        for a in [1..n+1]:
            term=sigma(a,k=j) 
            ans=ans+term*x^a
        return D(ans)


    def recipSigma(n):
        g=L(1/Sigma(2*n))
        return g

    def recipSigmaPowerConstant(n):
        f=recipSigma(n)^n
        return f[0]
    
    rsp=recipSigmaPowerConstant(2)
   
    o3=order(3,rsp)
    if mod(j,2)==1:
        data=data+[o3]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
for power in [1..10]:
    data=[]
    for j in [2..100]:
        def Sigma(n):
            ans=0
            for a in [1..n+1]:
                term=sigma(a,k=j) 
                ans=ans+term*x^a
            return D(ans)


        def recipSigma(n):
            g=L(1/Sigma(2*n))
            return g

        def recipSigmaPowerConstant(n):
            f=recipSigma(n)^n
            return f[0]
    
        rsp=recipSigmaPowerConstant(power)
   
        o3=order(3,rsp)
        if mod(j,2)==1:
            data=data+[o3]
    print("power",power);print(data)

power 1
[2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2, 1, 1, 2, 1, 1, 3]
power 2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
power 3
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
power 4
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
power 5
[1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1]
power 6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
power 7
[2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2

In [35]:
import pickle
rfile=open('/Users/barrybrent/run14may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

tpls=[]
for k in [0..199]:
    pair=s[k]
    a=pair[0]
    a3=ZZ(mod(a,3))
    if a3==1:
        ds2=digitsum(2,a)
        ds3=digitsum(3,a)
        c=pair[1]
        o2=order(2,c)
    print((a,ds2,ds3,o2))
    tpl=(ds2,ds3,o2)
    tpls=tpls+[tpl]
sst=sortAndSmash(tpls)
for tpl in sst:print(tpl)

(0, 5, 5, 3)
(1, 1, 1, 1)
(2, 1, 1, 1)
(3, 1, 1, 1)
(4, 1, 2, 3)
(5, 1, 2, 3)
(6, 1, 2, 3)
(7, 3, 3, 2)
(8, 3, 3, 2)
(9, 3, 3, 2)
(10, 2, 2, 3)
(11, 2, 2, 3)
(12, 2, 2, 3)
(13, 3, 3, 3)
(14, 3, 3, 3)
(15, 3, 3, 3)
(16, 1, 4, 3)
(17, 1, 4, 3)
(18, 1, 4, 3)
(19, 3, 3, 4)
(20, 3, 3, 4)
(21, 3, 3, 4)
(22, 3, 4, 6)
(23, 3, 4, 6)
(24, 3, 4, 6)
(25, 3, 5, 2)
(26, 3, 5, 2)
(27, 3, 5, 2)
(28, 3, 2, 5)
(29, 3, 2, 5)
(30, 3, 2, 5)
(31, 5, 3, 1)
(32, 5, 3, 1)
(33, 5, 3, 1)
(34, 2, 4, 2)
(35, 2, 4, 2)
(36, 2, 4, 2)
(37, 3, 3, 1)
(38, 3, 3, 1)
(39, 3, 3, 1)
(40, 2, 4, 3)
(41, 2, 4, 3)
(42, 2, 4, 3)
(43, 4, 5, 1)
(44, 4, 5, 1)
(45, 4, 5, 1)
(46, 4, 4, 2)
(47, 4, 4, 2)
(48, 4, 4, 2)
(49, 3, 5, 1)
(50, 3, 5, 1)
(51, 3, 5, 1)
(52, 3, 6, 3)
(53, 3, 6, 3)
(54, 3, 6, 3)
(55, 5, 3, 1)
(56, 5, 3, 1)
(57, 5, 3, 1)
(58, 4, 4, 2)
(59, 4, 4, 2)
(60, 4, 4, 2)
(61, 5, 5, 1)
(62, 5, 5, 1)
(63, 5, 5, 1)
(64, 1, 4, 3)
(65, 1, 4, 3)
(66, 1, 4, 3)
(67, 3, 5, 4)
(68, 3, 5, 4)
(69, 3, 5, 4)
(70, 3, 6, 3)
(71, 3, 6, 3)
(7